In [1]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
# Leitura dos arquivos
previsores = pd.read_csv('../input/entradas-breast/entradas_breast.csv')
classe = pd.read_csv('../input/saidas-breast/saidas_breast.csv')

In [3]:
previsores

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.12380,0.18660,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.00000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.13740,205.00000,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,111.00000,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.45,26.40,166.10,2027.0,141.00000,0.21130,0.4107,0.2216,206.0000,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,144.00000,0.09791,0.1752,0.05533,...,23.69,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,159.0000,0.05648,...,18.98,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,277.00000,0.35140,152.00000,0.2397,0.07016,...,25.74,39.42,184.60,1821.0,165.00000,0.86810,0.9387,265.0000,0.4087,124.00000


In [4]:
classe

,0
0,0
1,0
2,0
3,0
4,0
...,...
564,0
565,0
566,0
567,0


In [5]:
# Divisão entre treino e teste
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.25)

In [6]:
# Cria a rede neutal de camadas de modelo sequencial (camadas de entrada, camadas ocultas e camada de saída)
classificador = Sequential()

# Primeira camada oculta com 16 neurônios, liga a camada de entrada com 30 neurônios (30 atributos previsores)
# input_dim = só coloca na primeira camada oculta
# Função de ativação relu
# calculo neurônios da camada oculta = (30 (entrada) + 1 (saída)) / 2         ----> Resultado: 15.5
classificador.add(Dense(units = 16, activation = 'relu', kernel_initializer = 'random_uniform', input_dim = 30))

# Segunda camada oculta (com a mesma quantidade de neurônios da primeira camada de saída)
# Resultados piores com a adição da segunda camada oculta
classificador.add(Dense(units = 16, activation = 'relu', kernel_initializer = 'random_uniform'))

# Camada de saída com 1 neurônio (0 ou 1), problema de classificação binária
# função de ativação sigmoide (retorna um valor entre 0 e 1)
classificador.add(Dense(units = 1, activation = 'sigmoid'))

# Descida do gradiente (mínimo global)
# otimizador = Adam
# lr = taxa de aprendizagem (chegar ao mínimo global de forma mais rápida)
# decay = decaimento da taxa de aprendizagem, após a atualização dos pesos
# clipvalue = prender o valor de lr para que ele não fique muito disperso dentro do gradiente
otimizador = keras.optimizers.Adam(lr = 0.001, decay = 0.0001, clipvalue = 0.5)

# Execução da rede neural
# optimizer = Função utilizada para fazer o ajuste dos pesos da descida do gradiente
# loss = função de perda, forma como irá fazer o cálculo do erro. 
# binary_crossentropy = 2 classes para classificação (0 ou 1)
# categorical_crossentropy = mais de duas classes para classificação
# metrics = métrica utilizada para fazer a avaliação da classificação. Pode colocar mais de uma métrica
# binary_accuracy = acurácia para 2 classes (0 ou 1)
# Essa avaliação é feita no conjunto de treino
classificador.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])

#classificador.compile(optimizer = 'adam',  loss = 'binary_crossentropy',
#                      metrics = ['binary_accuracy'])

# treinamento
# batch_size = 10 --> calcula o erro para 10 registros e depois atualiza os pesos
# epochs = quantidade de épocas que irá executar o treinamento, ou seja, quantidade de vezes que irá fazer o ajuste dos pesos
classificador.fit(previsores_treinamento, classe_treinamento, batch_size = 10, epochs = 100)

Epoch 1/100
43/43 [==============================] - 1s 992us/step - loss: 0.8723 - binary_accuracy: 0.6264
Epoch 2/100
43/43 [==============================] - 0s 1ms/step - loss: 0.6052 - binary_accuracy: 0.7373
Epoch 3/100
43/43 [==============================] - 0s 913us/step - loss: 0.5183 - binary_accuracy: 0.7975
Epoch 4/100
43/43 [==============================] - 0s 921us/step - loss: 0.4293 - binary_accuracy: 0.8197
Epoch 5/100
43/43 [==============================] - 0s 980us/step - loss: 0.4128 - binary_accuracy: 0.8067
Epoch 6/100
43/43 [==============================] - 0s 966us/step - loss: 0.4352 - binary_accuracy: 0.8120
Epoch 7/100
43/43 [==============================] - 0s 940us/step - loss: 0.4441 - binary_accuracy: 0.7890
Epoch 8/100
43/43 [==============================] - 0s 936us/step - loss: 0.4144 - binary_accuracy: 0.8392
Epoch 9/100
43/43 [==============================] - 0s 998us/step - loss: 0.4061 - binary_accuracy: 0.8422
Epoch 10/100
43/43 [==========

In [7]:
# Pesos de cada camada da rede neural
pesos0 = classificador.layers[0].get_weights()
pesos1 = classificador.layers[1].get_weights()
pesos2 = classificador.layers[2].get_weights()

In [8]:
# Predição
previsoes = classificador.predict(previsores_teste)

# Limiar da predição para classificação em 0 ou 1
previsoes = (previsoes > 0.5) # > 0.5 --> 1

In [9]:
# Avaliação utilizando o sklearn (na base de dados de teste)
precisao = accuracy_score(classe_teste, previsoes) # Precisão na base de dados de teste
matriz = confusion_matrix(classe_teste, previsoes) # Matriz de confusão

# Avaliação utilizando o keras (na base de dados de teste)
resultado = classificador.evaluate(previsores_teste, classe_teste)

5/5 [==============================] - 0s 1ms/step - loss: 0.9993 - binary_accuracy: 0.8462


In [10]:
precisao

0.8461538461538461

In [11]:
matriz

array([[35, 18],
       [ 4, 86]])

In [12]:
resultado

[0.9992790222167969, 0.8461538553237915]